In [1]:
!pip install symspellpy fuzzywuzzy pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 16.2 MB/s eta 0:00:00


In [2]:
from spellchecker import SpellChecker
from fuzzywuzzy import process, fuzz
import os

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
# Load Word List (Correct Words Only)
def load_word_list(word_list_path):
    dictionary = []
    try:
        with open(word_list_path, "r", encoding="utf-8") as file:
            for line in file:
                dictionary.append(line.strip())
    except FileNotFoundError:
        print(f"Word list file '{word_list_path}' not found.")
        exit()
    return dictionary

#PySpellChecker

In [4]:
# Initialize PySpellChecker
def initialize_pyspellchecker():
    spell = SpellChecker(language=None)
    custom_word_list = "word_list.txt"
    if os.path.exists(custom_word_list):
        with open(custom_word_list, "r", encoding="utf-8") as file:
            words = file.read().splitlines()
            spell.word_frequency.load_words(words)
    return spell

# Correct Word Using PySpellChecker
def correct_word_pyspellchecker(spell, word):
    return spell.correction(word) or word

#FuzzyWuzzy

In [5]:
# Correct Word Using FuzzyWuzzy
def correct_word_fuzzywuzzy(word, dictionary):
    best_match = process.extractOne(word, dictionary, scorer=fuzz.ratio)
    return best_match[0] if best_match and best_match[1] > 80 else word

In [6]:
# Evaluate Accuracy of Each Method
def evaluate_accuracy(correct_word, ground_truth):
    return 1 if correct_word == ground_truth else 0

# Correct Word and Evaluate Accuracy
def correct_word_and_evaluate(word, ground_truth, spell, dictionary):
    corrected_pyspell = correct_word_pyspellchecker(spell, word)
    accuracy_pyspell = evaluate_accuracy(corrected_pyspell, ground_truth)

    corrected_fuzzywuzzy = correct_word_fuzzywuzzy(word, dictionary)
    accuracy_fuzzywuzzy = evaluate_accuracy(corrected_fuzzywuzzy, ground_truth)

    return corrected_pyspell, corrected_fuzzywuzzy, accuracy_pyspell, accuracy_fuzzywuzzy

In [7]:
# Correct and Evaluate Paragraph
def correct_paragraph(paragraph, spell, dictionary):
    words = paragraph.split()  # Split paragraph into words
    corrected_paragraph_pyspell = []
    corrected_paragraph_fuzzywuzzy = []
    total_accuracy = {"pyspell": 0, "fuzzywuzzy": 0}
    total_words = len(words)

    for word in words:
        # Find the closest match from the word list (ground truth)
        best_match = process.extractOne(word, dictionary, scorer=fuzz.ratio)
        if best_match and best_match[1] > 80:
            ground_truth = best_match[0]
        else:
            ground_truth = word  # If no close match, use the original word as ground truth

        corrected_pyspell, corrected_fuzzywuzzy, accuracy_pyspell, accuracy_fuzzywuzzy = correct_word_and_evaluate(
            word, ground_truth, spell, dictionary)

        corrected_paragraph_pyspell.append(corrected_pyspell)
        corrected_paragraph_fuzzywuzzy.append(corrected_fuzzywuzzy)

        total_accuracy["pyspell"] += accuracy_pyspell
        total_accuracy["fuzzywuzzy"] += accuracy_fuzzywuzzy

    return corrected_paragraph_pyspell, corrected_paragraph_fuzzywuzzy, total_accuracy, total_words

# Main Function
if __name__ == "__main__":
    # Initialize PySpellChecker
    pyspell = initialize_pyspellchecker()

    # Load dictionary from word list (correct words only)
    word_list_path = "word_list.txt"
    dictionary = load_word_list(word_list_path)

    # Ensure word list is not empty
    if not dictionary:
        print("No valid words found in the word list. Please check the file format.")
        exit()


In [ ]:
    # Input a paragraph
    paragraph_to_correct = input("Enter a paragraph: ").strip()

    # Correct the paragraph and evaluate accuracy
    corrected_paragraph_pyspell, corrected_paragraph_fuzzywuzzy, total_accuracy, total_words = correct_paragraph(
        paragraph_to_correct, pyspell, dictionary)

    # Print the corrected paragraphs
    print("\nCorrected Paragraph (PySpellChecker):")
    print(" ".join(corrected_paragraph_pyspell))

    print("\nCorrected Paragraph (FuzzyWuzzy):")
    print(" ".join(corrected_paragraph_fuzzywuzzy))

    # Final Accuracy Calculation
    print("\nFinal Accuracy:")
    print(f"PySpellChecker: {total_accuracy['pyspell'] / total_words * 100:.2f}%")
    print(f"FuzzyWuzzy: {total_accuracy['fuzzywuzzy'] / total_words * 100:.2f}%")

Enter a paragraph: පුවත්පත් කවි පිටු ගණනවක් සංස්කණයෙහි යෙදෙමින් ආධුනික කවීන්ට ගුරු උපදශ හා ධර්යය සපයන රත්න ශ්‍රී රූපවාහිනී හා ගුවන් විදුල වැඩසටහන් සම්පාකයකු ලෙස ද පරිචයක් ලැබුවේ ය.

Corrected Paragraph (PySpellChecker):
පුවත්පත් කවි පිටු ගණනාවක් සංස්කරණයෙහි යෙදෙමින් ආධුනික කවීන්ට ගුරු උපදේශ හා ධෛර්යය සපයන රත්න ශ්‍රී රූපවාහිනී හා ගුවන් විදුලි වැඩසටහන් සම්පාදකයකු ලෙස ද පරිචයක් ලැබුවේ ය

Corrected Paragraph (FuzzyWuzzy):
පුවත්පත් කවි පිටු ගණනාවක් සංස්කරණයෙහි යෙදෙමින් ආධුනික කවීන්ට ගුරු උපදේශ හා ධෛර්යය සපයන රත්න ශ්‍රී රූපවාහිනී හා ගුවන් විදුලි වැඩසටහන් සම්පාදකයකු ලෙස ද පරිචයක් ලැබුවේ ය

Final Accuracy:
PySpellChecker: 100.00%
FuzzyWuzzy: 100.00%


In [ ]:
    # Input a paragraph
    paragraph_to_correct = input("Enter a paragraph: ").strip()

    # Correct the paragraph and evaluate accuracy
    corrected_paragraph_pyspell, corrected_paragraph_fuzzywuzzy, total_accuracy, total_words = correct_paragraph(
        paragraph_to_correct, pyspell, dictionary)

    # Print the corrected paragraphs
    print("\nCorrected Paragraph (PySpellChecker):")
    print(" ".join(corrected_paragraph_pyspell))

    print("\nCorrected Paragraph (FuzzyWuzzy):")
    print(" ".join(corrected_paragraph_fuzzywuzzy))

    # Final Accuracy Calculation
    print("\nFinal Accuracy:")
    print(f"PySpellChecker: {total_accuracy['pyspell'] / total_words * 100:.2f}%")
    print(f"FuzzyWuzzy: {total_accuracy['fuzzywuzzy'] / total_words * 100:.2f}%")

Enter a paragraph: සීතල උඳුප් මහේ සාමයේ කුමරන්ගේ උපත පිළිබඳ පණිවුඩය රැගෙන මඳ නල දසත හමා යද්දී උදා වූ අසිරිමත් නත්තලෙහි ප්‍රීතයෙන් මුළු ලව ම නැහවී ගියේය.

Corrected Paragraph (PySpellChecker):
සීතල උඳුවප් මහේ සාමයේ කුමරුන්ගේ උපත පිළිබඳ පණිවුඩය රැගෙන මඳ නල දසත හමා යද්දී උදා වූ අසිරිමත් නත්තලෙහි ප්‍රීතියෙන් මුළු නව ම නැහැවී ගියේය

Corrected Paragraph (FuzzyWuzzy):
සීතල උඳුවප් මහේ සාමයේ කුමරුන්ගේ උපත පිළිබඳ පණිවුඩය රැගෙන මඳ නල දසත හමා යද්දී උදා වා අසිරිමත් නත්තලෙහි ප්‍රීතියෙන් මුළු ලව ම නැහැවී ගියේය

Final Accuracy:
PySpellChecker: 91.67%
FuzzyWuzzy: 100.00%


In [ ]:
    # Input a paragraph
    paragraph_to_correct = input("Enter a paragraph: ").strip()

    # Correct the paragraph and evaluate accuracy
    corrected_paragraph_pyspell, corrected_paragraph_fuzzywuzzy, total_accuracy, total_words = correct_paragraph(
        paragraph_to_correct, pyspell, dictionary)

    # Print the corrected paragraphs
    print("\nCorrected Paragraph (PySpellChecker):")
    print(" ".join(corrected_paragraph_pyspell))

    print("\nCorrected Paragraph (FuzzyWuzzy):")
    print(" ".join(corrected_paragraph_fuzzywuzzy))

    # Final Accuracy Calculation
    print("\nFinal Accuracy:")
    print(f"PySpellChecker: {total_accuracy['pyspell'] / total_words * 100:.2f}%")
    print(f"FuzzyWuzzy: {total_accuracy['fuzzywuzzy'] / total_words * 100:.2f}%")

Enter a paragraph: ප්‍රථමික අධ්‍යාපනය අවසන් කළ කුඩා එඩ්න්ඩ් පීරිස් කළඹ සාන්ත ජෝසප් විද්‍යායට ඇතුළත්ව එහි දීප්තිමත් ශිෂ්‍යකු වශයෙන් උසස් ලෙස සමත්කම් දක්වමින් ද්විතයක අධ්‍යනය නිම කරයි.

Corrected Paragraph (PySpellChecker):
ප්‍රාථමික අධ්‍යාපනය අවසන් කළ කුඩා එඩ්මන්ඩ් පීරිස් කළ සාන්ත ජෝසප් විද්‍යාලයට ඇතුළත්ව එහි දීප්තිමත් ශිෂ්‍යයකු වශයෙන් උසස් ලෙස සමත්කම් දක්වමින් ද්විතීයක අධ්‍යාපනය නිම කරයි.

Corrected Paragraph (FuzzyWuzzy):
ප්‍රාථමික අධ්‍යාපනය අවසන් කළ කුඩා එඩ්මන්ඩ් පීරිස් කොළඹ සාන්ත ජෝසප් විද්‍යාලයට ඇතුළත්ව එහි දීප්තිමත් ශිෂ්‍යයකු වශයෙන් උසස් ලෙස සමත්කම් දක්වමින් ද්විතීයක අධ්‍යාපනය නිම කරයි

Final Accuracy:
PySpellChecker: 91.67%
FuzzyWuzzy: 100.00%


In [ ]:
    # Input a paragraph
    paragraph_to_correct = input("Enter a paragraph: ").strip()

    # Correct the paragraph and evaluate accuracy
    corrected_paragraph_pyspell, corrected_paragraph_fuzzywuzzy, total_accuracy, total_words = correct_paragraph(
        paragraph_to_correct, pyspell, dictionary)

    # Print the corrected paragraphs
    print("\nCorrected Paragraph (PySpellChecker):")
    print(" ".join(corrected_paragraph_pyspell))

    print("\nCorrected Paragraph (FuzzyWuzzy):")
    print(" ".join(corrected_paragraph_fuzzywuzzy))

    # Final Accuracy Calculation
    print("\nFinal Accuracy:")
    print(f"PySpellChecker: {total_accuracy['pyspell'] / total_words * 100:.2f}%")
    print(f"FuzzyWuzzy: {total_accuracy['fuzzywuzzy'] / total_words * 100:.2f}%")

Enter a paragraph: සියනෑ කෝරළයේ කිරිඳිවල නගයට ආසන්න රදාවන ගම් පියසෙහි දී මහගමසේකර මහත උපන්නේය. ඔහු පස කලෙක සිය නිර්මණ ජිවයට එම අත්දැකීම් පිටිවහලක් කර ගත්තේය.

Corrected Paragraph (PySpellChecker):
සියනෑ කෝරළයේ කිරිඳිවැල නගරයට ආසන්න රදාවාන ගම් පියසෙහි දී මහගමසේකර මහතා උපන්නේය ඔහු පසු කලෙක සිය නිර්මාණ ජිවිතයට එම අත්දැකීම් පිටිවහලක් කර ගත්තේය

Corrected Paragraph (FuzzyWuzzy):
සියනෑ කෝරළයේ කිරිඳිවැල නගරයට ආසන්න රදාවාන ගම් පියසෙහි ද මහගමසේකර මහතා උපන්නේය ඔහු පසු කලෙක සිය නිර්මාණ ජිවිතයට එම අත්දැකීම් පිටිවහලක් කර ගත්තාය

Final Accuracy:
PySpellChecker: 91.30%
FuzzyWuzzy: 100.00%


In [ ]:
    # Input a paragraph
    paragraph_to_correct = input("Enter a paragraph: ").strip()

    # Correct the paragraph and evaluate accuracy
    corrected_paragraph_pyspell, corrected_paragraph_fuzzywuzzy, total_accuracy, total_words = correct_paragraph(
        paragraph_to_correct, pyspell, dictionary)

    # Print the corrected paragraphs
    print("\nCorrected Paragraph (PySpellChecker):")
    print(" ".join(corrected_paragraph_pyspell))

    print("\nCorrected Paragraph (FuzzyWuzzy):")
    print(" ".join(corrected_paragraph_fuzzywuzzy))

    # Final Accuracy Calculation
    print("\nFinal Accuracy:")
    print(f"PySpellChecker: {total_accuracy['pyspell'] / total_words * 100:.2f}%")
    print(f"FuzzyWuzzy: {total_accuracy['fuzzywuzzy'] / total_words * 100:.2f}%")

Enter a paragraph: මිනිසුන් දෙදනෙකු මටර් රථ ක්‍රීඩවක යෙදෙයි. තරඟකරුවෙක් අනකා හඹා යයි.  මිනිසෙක් ධානය වන මොටර් රයක එල්ල  සිටිය.

Corrected Paragraph (PySpellChecker):
මිනිසුන් දෙදෙනෙකු මොටර් රථ ක්‍රීඩාවක යෙදෙයි. තරඟකරුවෙක් අනෙකා හඹා යයි. මිනිසෙක් ධාවනය වන මොටර් රථයක එල්ල සිටියි

Corrected Paragraph (FuzzyWuzzy):
මිනිසුන් දෙදෙනෙක් මොටර් රථ ක්‍රීඩාවක යෙදෙයි. තරඟකරුවෙක් අනෙකා හඹා යයි මිනිසෙක් ධාවනය වන මොටර් රථයක එල්ල සිටියි

Final Accuracy:
PySpellChecker: 88.24%
FuzzyWuzzy: 100.00%
